In [128]:
import pandas as pd
import numpy as np

from os import listdir
from os.path import join

In [129]:
data_path = 'data'

In [130]:
onlyfiles = [f for f in listdir(data_path) if f.split('.')[-1]=='csv']

In [138]:
def read_data(file_name):
    data = pd.read_csv(file_name, index_col=False,
                       usecols = ['High Load Start', 'High Load Finish', 'utility', 'date', 'holiday'])
    # clean up the data
    data = data[data['utility']!='epb']      # exclude utility of epb
    data = data.dropna()
    data = data.rename(columns={"High Load Start": "start_time", "High Load Finish": "final_time"})
    data = data[(data['start_time']>=4)&(data['start_time']<=12)]
    return data

In [133]:
def random_to_15_mins(data):
    '''randomly smooth out to 15 mins
    '''
    start_time_noise = np.random.random(len(data))-0.5
    final_time_noise = np.random.random(len(data))-0.5
    data['start_time'] = np.round((data['start_time'].values+start_time_noise)*4)/4
    data['final_time'] = np.round((data['final_time'].values+final_time_noise)*4)/4
    return data

In [169]:
def sliding_window_filter(dataframe, filter_length=16):
    '''
    Using a rolling 4-h window, the slot with the maximum sum of marginal distributions is identified for both 
    start times.
    For each row, filter out a successive series of data with the length of filter_length that has the maximum sum  
    '''
    
    # generate filter matrix
    row_length = dataframe.shape[1]
    filter_matrix_list = []
    for row_id in range(row_length):
        row = [0]*row_id + [1]*min(filter_length, row_length-row_id) + [0]*max(0, row_length-row_id-filter_length)
        filter_matrix_list.append(row)
    filter_matrix = np.array(filter_matrix_list).T
    
    # filter each row of the dataframe
    for row_id in range(dataframe.shape[0]):
        row = dataframe.iloc[row_id].values
        accuSum = np.dot(row,filter_matrix) # If a is an N-D array and b is a 1-D array, 
                                            # it is a sum product over the last axis of a (column axis) and b.
        start_index = accuSum.argmax()
        filter_row = [0]*start_index + [1]*min(filter_length, row_length-start_index) + \
                     [0]*max(0, row_length-start_index-filter_length)
        filtered = row*filter_row
        filtered = filtered/filtered.sum()
        dataframe.iloc[row_id] = filtered
    
    return dataframe

In [170]:
def generate_distribution(data, building_type, slidingWindowFilter=False):
    '''
    Input has two columns: start_time, duration
    Output: two distributions: start_time distribution, duration distribution
    '''
    start_time_count_list = []
    duration_freq_all_df_raw = pd.DataFrame()
    for start_time in np.arange(3.5,12.5,0.25):
        duration_unique, duration_counts = np.unique(data[data['start_time']==start_time]['duration'], 
                                                     return_counts=True)
        start_time_count = duration_counts.sum()
        start_time_count_list.append(start_time_count)
        duration_freq = duration_counts/start_time_count

        duration_freq_df = pd.DataFrame(data=duration_freq, index=duration_unique)
        duration_freq_all_df_raw = pd.concat([duration_freq_all_df_raw, duration_freq_df], axis=1)

    start_time_df = pd.DataFrame(data=start_time_count_list/np.array(start_time_count_list).sum(), 
                                 index=['Option={}'.format(time) for time in np.arange(3.5,12.5,0.25)], 
                                 columns=[building_type])
    start_time_df = start_time_df.T

    duration_freq_all_df_raw.fillna(0, inplace=True)
    duration_freq_all_df_raw = duration_freq_all_df_raw.reset_index().set_index('index').T.reset_index().drop(columns=['index'])

    duration_freq_all_df = pd.DataFrame()
    duration_range = np.arange(5.75, 19, 0.25)
    for duration in duration_range:
        if duration in duration_freq_all_df_raw.columns:
            duration_freq_all_df['Option={}'.format(duration)] = duration_freq_all_df_raw[duration]
        else:
            duration_freq_all_df['Option={}'.format(duration)] = 0

    rowSum = duration_freq_all_df.sum(axis=1).values
    rowSum = rowSum[...,np.newaxis]
    duration_freq_all_df = pd.DataFrame(duration_freq_all_df.values/rowSum)
    duration_freq_all_df.columns = ['Option={}'.format(duration) for duration in duration_range]

    if slidingWindowFilter:
        start_time_df = sliding_window_filter(start_time_df, filter_length=16)
        duration_freq_all_df = sliding_window_filter(duration_freq_all_df, filter_length=16)
    
    duration_freq_all_df.insert(0, 'Dependency=building_type', building_type)
    duration_freq_all_df.insert(1, 'Dependency=start_time', np.arange(3.5,12.5,0.25))
    duration_freq_all_df.fillna(0, inplace=True)
    
    start_time_df = start_time_df.round(3)
    duration_freq_all_df = duration_freq_all_df.round(3)
        
    return start_time_df, duration_freq_all_df

## Generate without filtering

In [145]:
start_time_WD_all = pd.DataFrame()
duration_freq_WD_all = pd.DataFrame()
start_time_NWD_all = pd.DataFrame()
duration_freq_NWD_all = pd.DataFrame()

for file in onlyfiles:
    building_type = file.split('.')[0].split('-')[1]
    print('Processing {}'.format(building_type))
    data = read_data(join(data_path, file))
    data = random_to_15_mins(data)
    data['duration'] = data['final_time'] - data['start_time']
    
    # working day vs non-working day
    data['time'] = pd.to_datetime(data['date'])
    data['workingDay'] = data.apply(lambda x: (x['time'].weekday()<5)&(not x['holiday']), axis=1)  # axis=1 must be there
    data_WD = data[data['workingDay']]
    data_NWD = data[~ data['workingDay']]    

    # generate distribution
    start_time_WD, duration_freq_WD = generate_distribution(data_WD, building_type)
    start_time_NWD, duration_freq_NWD = generate_distribution(data_NWD, building_type) 

    # merge into one file
    start_time_WD_all = pd.concat([start_time_WD_all, start_time_WD], axis=0)
    start_time_NWD_all = pd.concat([start_time_NWD_all, start_time_NWD], axis=0)
    
    duration_freq_WD_all = pd.concat([duration_freq_WD_all, duration_freq_WD], axis=0)
    duration_freq_NWD_all = pd.concat([duration_freq_NWD_all, duration_freq_NWD], axis=0)   

Processing auto_dealership
Processing auto_repair
Processing car_wash


C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


Processing drug_store
Processing full_service_restaurant
Processing gas_station_or_convenience_store
Processing hospital


C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


Processing house_of_worship
Processing large_hotel
Processing large_office
Processing library


...

C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


Processing small_hotel
Processing small_office
Processing strip_mall
Processing supermarket


C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


Processing warehouse
Processing warehouse_refrigerated


C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


In [160]:
start_time_WD_all.index.name = 'Dependency=building_type'
start_time_NWD_all.index.name = 'Dependency=building_type'
start_time_WD_all.to_csv('distribution/notFiltered/weekday_start_time.tsv', sep = '\t')
start_time_NWD_all.to_csv('distribution/notFiltered/weekend_start_time.tsv', sep = '\t')

In [161]:
duration_freq_WD_all.fillna(0, inplace=True)
duration_freq_NWD_all.fillna(0, inplace=True)
duration_freq_WD_all.to_csv('distribution/notFiltered/weekday_duration.tsv', sep = '\t', index=False)
duration_freq_NWD_all.to_csv('distribution/notFiltered/weekend_duration.tsv', sep = '\t', index=False)

## Generate without filtering

In [171]:
start_time_WD_all = pd.DataFrame()
duration_freq_WD_all = pd.DataFrame()
start_time_NWD_all = pd.DataFrame()
duration_freq_NWD_all = pd.DataFrame()

for file in onlyfiles:
    building_type = file.split('.')[0].split('-')[1]
    print('Processing {}'.format(building_type))
    data = read_data(join(data_path, file))
    data = random_to_15_mins(data)
    data['duration'] = data['final_time'] - data['start_time']
    
    # working day vs non-working day
    data['time'] = pd.to_datetime(data['date'])
    data['workingDay'] = data.apply(lambda x: (x['time'].weekday()<5)&(not x['holiday']), axis=1)  # axis=1 must be there
    data_WD = data[data['workingDay']]
    data_NWD = data[~ data['workingDay']]    

    # generate distribution
    start_time_WD, duration_freq_WD = generate_distribution(data_WD, building_type, slidingWindowFilter=True)
    start_time_NWD, duration_freq_NWD = generate_distribution(data_NWD, building_type, slidingWindowFilter=True) 

    # merge into one file
    start_time_WD_all = pd.concat([start_time_WD_all, start_time_WD], axis=0)
    start_time_NWD_all = pd.concat([start_time_NWD_all, start_time_NWD], axis=0)
    
    duration_freq_WD_all = pd.concat([duration_freq_WD_all, duration_freq_WD], axis=0)
    duration_freq_NWD_all = pd.concat([duration_freq_NWD_all, duration_freq_NWD], axis=0)   

Processing auto_dealership
Processing auto_repair
Processing car_wash


C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


Processing drug_store
Processing full_service_restaurant
Processing gas_station_or_convenience_store
Processing hospital


C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


Processing house_of_worship
Processing large_hotel
Processing large_office
Processing library


...

C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


Processing small_hotel
Processing small_office
Processing strip_mall
Processing supermarket


C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


Processing warehouse
Processing warehouse_refrigerated


C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
C:\Users\hongteam\Anaconda3\envs\eulp\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide


In [172]:
start_time_WD_all.index.name = 'Dependency=building_type'
start_time_NWD_all.index.name = 'Dependency=building_type'
start_time_WD_all.to_csv('distribution/filtered/weekday_start_time_filtered.tsv', sep = '\t')
start_time_NWD_all.to_csv('distribution/filtered/weekend_start_time_filtered.tsv', sep = '\t')

In [173]:
duration_freq_WD_all.fillna(0, inplace=True)
duration_freq_NWD_all.fillna(0, inplace=True)
duration_freq_WD_all.to_csv('distribution/filtered/weekday_duration_filtered.tsv', sep = '\t', index=False)
duration_freq_NWD_all.to_csv('distribution/filtered/weekend_duration_filtered.tsv', sep = '\t', index=False)